In [315]:
# We are expecting that we can get the flow of the code . In the mean time we can use a code which is by default in flow...
import ast

with open('test.py', 'r') as f:
    code = f.read()

tree = ast.parse(code)

with open ('modules.txt', 'w') as file:  
    file.write(ast.dump(tree, indent=1)) 


In [316]:
node_list=[]
class MyVisitor(ast.NodeVisitor):
    def visit(self, node):
        node_list.append(node)
        """Visit a node."""
        method = 'visit_' + node.__class__.__name__
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node)
    
v=MyVisitor()
v.visit(tree)
for node in node_list:
    print(node.__class__.__name__)

Module
Assign
Name
Store
Constant
Assign
Name
Store
Call
Name
Load
Call
Name
Load
Constant
While
Compare
Name
Load
Lt
Name
Load
AugAssign
Name
Store
Mult
Constant
AugAssign
Name
Store
Add
Constant
AugAssign
Name
Store
Mult
Constant
Assign
Name
Store
Constant
While
Compare
Name
Load
Lt
Name
Load
AugAssign
Name
Store
Add
Constant
Expr
Call
Name
Load
Constant
Name
Load


In [317]:
# Time Complexities 

# O(1) - Constant time
# O(n) - Linear time
# O(n^2) - Quadratic time
# O(n^c) - Polynomial time
# O(log n) - Logarithmic time
# O(n log n) - Linearithmic
# O(2^n) - Exponential time
# O(n!) - Factorial time

## Cleaning the intermediate code

In [318]:
# Remove print statements from the intermediate code

refactored_tree = []

for node in ast.walk(tree):
    if not (isinstance(node, ast.Call) and isinstance(node.func, ast.Name) and node.func.id == "print"):
        #print(ast.dump(node))
        refactored_tree.append(node)
        #print(" ")


## Important Functionalities 

In [319]:
# Check if the given variable is really a variable or is a constant

import re

def checkVar(name,nodes_visited):

    nodes_visited.reverse()

    for node in nodes_visited:
        if node.__class__.__name__=='Assign':

            var=node.targets[0]
            values=node.value
            
            if var.id==name:
                    
                if isinstance(values, ast.Call) and isinstance(values.func, ast.Name) and (values.func.id == "input" or values.func.id == "int" or values.func.id == "float" or values.func.id == "double") :
                    return True
                
                elif isinstance(values, ast.Name):
                    return checkVar(values.id)
                
                elif isinstance(values, ast.BinOp) :
                    parse_it=ast.dump(values)
                    #print(type(parse_it))
                    ids = re.findall(r"id='(.*?)'", parse_it)
                    bool_upto_now=False
                    for all_var in ids:
                        bool_upto_now = bool_upto_now or checkVar(all_var,nodes_visited)
                    return bool_upto_now
            
        elif node.__class__.__name__=='AugAssign':

            var=node.target
            values=node.value

            if var.id==name:
                    
                if isinstance(values, ast.Call) and isinstance(values.func, ast.Name) and (values.func.id == "input" or values.func.id == "int" or values.func.id == "float" or values.func.id == "double") :
                    return True
                
                elif isinstance(values, ast.BinOp) :
                    parse_it=ast.dump(values)
                    #print(type(parse_it))
                    
                    ids = re.findall(r"id='(.*?)'", parse_it)
                    bool_upto_now=False
                    for all_var in ids:
                        bool_upto_now = bool_upto_now or checkVar(all_var,nodes_visited)
                    return bool_upto_now
                
        
                
    return False
                
                

## Priorities

In [320]:
def pref(a):
    a=a[0]

    if a == 'C' :
        return 0
    elif a == 'A'or a=='S' :
        return 1
    
    elif a=='M' or a=='D':
        return 2
        

def order_pref(change_left,change_right,leftVar,rightVar,nodes_visited):


    if pref(change_left)==pref(change_right):
        if checkVar(leftVar,nodes_visited):
            return change_left,leftVar
        
        elif checkVar(rightVar,nodes_visited):
            return change_right,rightVar
        
        else :
            return 'C',""
    
    elif pref(change_left)>pref(change_right):

            return change_left,rightVar
        
    elif pref(change_left)<pref(change_right):
    
            return change_right,leftVar




def complexity_pref(a,b):
    if a is None:
        return b
    elif b is None :
        return a 

    if '+' in a:
        a=(a.split("+"))[0]
    if '+' in b:
        b=(b.split("+"))[0]
    a_m=a.count("*")
    b_m=b.count("*")
    a_ln=a.count("ln")
    b_ln=b.count("ln")
    a_diff=a_m-a_ln
    b_diff=b_m-b_ln

    print(a, b)

    if a_diff>b_diff :
        return a 
    
    elif a_diff < b_diff :
        return b
    
    else :
        if a_ln>b_ln :
            return a
    
        elif a_ln<b_ln :
            return b 
        
        else :
            return '('+a+'+'+b+')'

## Parsing all nodes 

In [323]:
# Parsing All nodes
time_complexity='C'
def visit_All(tree_list,nodes_visited,left_Variables,right_Variables):
        global time_complexity
        v=ast.NodeVisitor()
        
        change_in_left={}
        change_in_right={}

        for node in tree_list:
                nodes_visited.append(node)

                # While loop has 3 parts test ,body , or else  

                if type(node).__name__ == 'While':
                        test=node.test
                        body=node.body
                        leftVar=""
                        rightVar=""
                        left_Status=False
                        right_Status=False

                        if isinstance(test, ast.Compare):
                                left = test.left
                                right = test.comparators[0]
                                op = test.ops[0]

                                if isinstance(left, ast.Name):
                                        leftVar=left.id
                                
                                left_Status=checkVar(node,nodes_visited)
                                #What if is it like while i<len(num_list)
                                if isinstance(right, ast.Call) and right.func.id == "len":
                                        print("")


                                # What if is it like while i<n 
                                elif isinstance(right, ast.Name) :
                                        rightVar=right.id
                                        right_Status=checkVar(right.id,nodes_visited)
                                        print(right_Status)

                                if right_Status==True or left_Status==True:
                                        if left_Variables.get(leftVar) is None:
                                                left_Variables[leftVar]='C'
                                        if right_Variables.get(rightVar) is None :
                                                right_Variables[rightVar]='C'

                        int_time=visit_All(body,nodes_visited,left_Variables,right_Variables)
                        #print(int_time)

                        change_left='C'
                        change_right='C'

                        if left_Variables.get(leftVar) is not None:

                                change_left=left_Variables.get(leftVar)

                                print(change_left," ",change_right)

                        if right_Variables.get(rightVar) is not None:
                
                                change_right=right_Variables.get(rightVar)

                                print(change_left," ",change_right)

                        preference,dominating=order_pref(change_left,change_right,leftVar,rightVar,nodes_visited)
                        print(preference, dominating, "2")
                        if preference=='C' or checkVar(dominating,nodes_visited) == False  :
                                time_complexity=complexity_pref(time_complexity,int_time)
                                        
                        elif preference == 'A' or preference =='S' :
                                time_complexity=complexity_pref(time_complexity,dominating+'*'+int_time)

                        else:
                                number=preference[2:]
                                time_complexity=complexity_pref(time_complexity,'ln'+number+'('+dominating+')'+'*'+int_time)


                elif type(node).__name__ =='For':
                        print("")

                elif type(node).__name__ =='Assign':

                        var=node.targets[0]
                        values=node.value
                        if isinstance(values,ast.Constant) :
                                left_Variables[var.id]='C'
                                continue
                        if isinstance(values, ast.Call) and isinstance(values.func, ast.Name) and (values.func.id == "input" or values.func.id == "int" or values.func.id == "float" or values.func.id == "double") :
                                left_Variables[var.id]='C'
                                continue
                        parse_it=ast.dump(values)
                        print(parse_it)
                        print(node.lineno, node.end_lineno)
                        ope = re.findall(r"op=([A-Za-z]+)\(\)", parse_it)
                        print(ope)
                        ope=ope[0]
                        if left_Variables.get(var.id) is not None :
                                act=left_Variables.get(var.id)
                                if ope=='A':
                                        if pref(act)<1 :
                                                left_Variables[var.id]='A'
                                elif ope=='S':
                                        if pref(act)<1 :
                                                left_Variables[var.id]='S'

                                elif ope=='M':
                                        ids = re.findall(r"id='(.*?)'", parse_it)
                                        num=int(re.findall(r"value='(.*?)'", parse_it))
                                        if pref(act)<2 :
                                                left_Variables[var.id]='M_'+str(num)
                                        else :
                                                if left_Variables[var.id][0]=='D':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='D_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='M':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='D_'+str(num_now_rev)

                                        
                                elif ope=='D':
                                        parse_it=ast.dump(values)
                                        num = int(re.findall(r"value='(.*?)'", parse_it))
                                        if pref(act)<2 :
                                                left_Variables[var.id]='D_'+str(num)
                                        else :
                                                if left_Variables[var.id][0]=='M':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='M_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='D':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='M_'+str(num_now_rev)



                        if right_Variables.get(var.id) is not None :
                                act=right_Variables.get(var.id)
                                if ope=='A':
                                        if pref(act)<1 :
                                                right_Variables[var.id]='A'
                                elif ope=='S' :
                                        if pref(act)<1 :
                                                right_Variables[var.id]='S'

                                elif ope=='M':
                                        num = int(re.findall(r"value='(.*?)'", parse_it))
                                        if pref(act)<2 :
                                                right_Variables[var.id]='M_'+str(num)
                                        else :
                                                if right_Variables[var.id][0]=='D':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='D_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='M':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='D_'+str(num_now_rev)

                                        
                                elif ope=='D':
                                        num = int(re.findall(r"value='(.*?)'", parse_it))
                                        if pref(act)<2 :
                                                right_Variables[var.id]='D_'+str(num)
                                        else :
                                                if right_Variables[var.id][0]=='M':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='M_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='D':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='M_'+str(num_now_rev)
                        
                
                elif type(node).__name__ =='AugAssign':
                        var=node.target
                        values=node.value
                        
                        op=node.op
                        
                        print(var.id, "1", values, op.__class__.__name__)
                        if left_Variables.get(var.id) is not None :
                                act=left_Variables.get(var.id)
                                print(act)
                                if op.__class__.__name__=='Add' :
                                        print("Bow")
                                        if pref(act)<1 :
                                                left_Variables[var.id]='A'
                                elif op.__class__.__name__=='Sub' :
                                        if pref(act)<1 :
                                                left_Variables[var.id]='S'

                                elif op.__class__.__name__=='Mult':
                                        parse_it=ast.dump(values)
                                        lop = re.findall(r"value=([0-9]+)\)", parse_it)
                                        
                                        num = int(lop[0])
                                        if pref(act)<2 :
                                                left_Variables[var.id]='M_'+str(num)
                                        else :
                                                if left_Variables[var.id][0]=='D':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='D_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='M':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='D_'+str(num_now_rev)

                                        
                                elif op.__class__.__name__=='Div':
                                        parse_it=ast.dump(values)
                                        lop = re.findall(r"value=([0-9]+)\)", parse_it)
                                        num = int(lop[0])
                                        if pref(act)<2 :
                                                left_Variables[var.id]='D_'+str(num)
                                        else :
                                                if left_Variables[var.id][0]=='M':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='M_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='D':
                                                        act_num=int(left_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                left_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                left_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                left_Variables[var.id]='M_'+str(num_now_rev)
                                



                        if right_Variables.get(var.id) is not None :
                                act=right_Variables.get(var.id)
                                if op.__class__.__name__=='Add' :
                                        if pref(act)<1 :
                                                right_Variables[var.id]='A'
                                elif op.__class__.__name__=='Sub' :
                                        if pref(act)<1 :
                                                right_Variables[var.id]='S'

                                elif op.__class__.__name__=='Mult':
                                        parse_it=ast.dump(values)
                                        lop = re.findall(r"value=([0-9]+)\)", parse_it)
                                        num = int(lop[0])
                                        if pref(act)<2 :
                                                right_Variables[var.id]='M_'+str(num)
                                        else :
                                                if right_Variables[var.id][0]=='D':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='D_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='M':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='M_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='D_'+str(num_now_rev)

                                        
                                elif op.__class__.__name__=='Div':
                                        parse_it=ast.dump(values)
                                        lop = re.findall(r"value=([0-9]+)\)", parse_it)
                                        
                                        num = int(lop[0])
                                        if pref(act)<2 :
                                                right_Variables[var.id]='D_'+str(num)
                                        else :
                                                if right_Variables[var.id][0]=='M':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num/act_num
                                                        num_now_rev=act_num/num

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='M_'+str(num_now_rev)

                                                elif left_Variables[var.id][0]=='D':
                                                        act_num=int(right_Variables[var.id][2:])
                                                        num_now=num*act_num
                                                        num_now_rev=int(1/num_now)

                                                        if num_now ==1 :
                                                                right_Variables[var.id]='C'
                                                        elif num_now>1 :
                                                                right_Variables[var.id]='D_'+str(num_now)
                                                        elif num_now<1 :
                                                                right_Variables[var.id]='M_'+str(num_now_rev)
                                #print(right_Variables)
                        
                        

                elif type(node).__name__ =='Exec':
                        print("") 


        return time_complexity
                

if __name__ == '__main__':
        print(visit_All(node_list, [], {}, {}))
                        
                
        
                




True
i 1 <ast.Constant object at 0x118cdfca0> Mult
C
i 1 <ast.Constant object at 0x118cdfc10> Add
M_23
Bow
i 1 <ast.Constant object at 0x118cdf880> Mult
M_23
True
j 1 <ast.Constant object at 0x118e7fbb0> Add
C
Bow
A   C
A   C
A n 2
C n*C
M_69   C
M_69   C
M_69 n 2
n*C ln69(n)*n*C
i 1 <ast.Constant object at 0x118cdfca0> Mult
M_69
i 1 <ast.Constant object at 0x118cdfc10> Add
M_1587
Bow
i 1 <ast.Constant object at 0x118cdf880> Mult
M_1587
True
j 1 <ast.Constant object at 0x118e7fbb0> Add
C
Bow
A   C
A   C
A n 2
ln69(n)*n*C n*ln69(n)*n*C
j 1 <ast.Constant object at 0x118e7fbb0> Add
A
Bow
n*ln69(n)*n*C
